# Processing data

## **Introduction**

In this chapter, we will introduce the use of data in JSON, CSV, and XML formats. This will include the means of parsing and converting this data to other formats, including storing that data in relational databases, search engines such as Elasticsearch, and cloud storage including AWS S3. We will also discuss the creation of distributed and large-scale scraping tasks through the use of messaging systems including AWS Simple Queue Service (SQS).  The goal is to provide both an understanding of the various forms of data you may retrieve and need to parse, and an instruction the the various backends where you can store the data you have scraped.  Finally, we get a first introduction to one and Amazon Web Service (AWS) offerings.  By the end of the book we will be getting quite heavy into AWS and this gives a gentle introduction.

## **Working with CSV and JSON data**

Extracting data from HTML pages is done using the techniques in the previous chapter, primarily using XPath through various tools and also with Beautiful Soup. While we will focus primarily on HTML, HTML is a variant of XML (eXtensible Markup Language).  XML one was the most popular for  of expressing data on the web, but other have become popular, and even exceeded XML in popularity. 

Two common formats that you will see are **JSON (JavaScript Object Notation)** and **CSV (Comma Separated Values)**.  CSV is easy to create and a common form for many spreadsheet applications, so many web sites provide data in that for, or you will need to convert scraped data to that format for further storage or collaboration. JSON really has become the preferred format, due to its easy within programming languages such as JavaScript (and Python), and many database now support it as a native data format.

In this recipe let's examine converting scraped data to CSV and JSON, as well as writing the data to files and also reading those data files from remote servers. The tools we will examine are the Python CSV and JSON libraries. We will also examine using pandas for these techniques.

> Also implicit in these examples is the conversion of XML data to CSV and JSON, so we won't have a dedicated section for those examples.

### **Getting ready**

We will be using the planets data page and converting that data into CSV and JSON files. Let's start by loading the planets data from the page into a list of python dictionary objects. 

In [7]:
import requests
from bs4 import BeautifulSoup

def get_planet_data():
    html = requests.get("http://localhost:8080/planets.html").text
    soup = BeautifulSoup(html, "lxml")

    planets_trs = soup.html.body.div.table.findAll("tr", {"class": "planet"})

    def to_dict(tr):
        tds = tr.findAll("td")
        planet_data = dict()
        planet_data['Name'] = tds[1].text.strip()
        planet_data['Mass'] = tds[2].text.strip()
        planet_data['Radius'] = tds[3].text.strip()
        planet_data['Description'] = tds[4].text.strip()
        planet_data['MoreInfo'] = tds[5].findAll("a")[0]["href"].strip()
        return planet_data
    
    planets = [to_dict(tr) for tr in planets_trs]
    return planets

if __name__ == "__main__":
    print(get_planet_data())

[{'Name': 'Mercury', 'Mass': '0.330', 'Radius': '4879', 'Description': 'Named Mercurius by the Romans because it appears to move so swiftly.', 'MoreInfo': 'https://en.wikipedia.org/wiki/Mercury_(planet)'}, {'Name': 'Venus', 'Mass': '4.87', 'Radius': '12104', 'Description': 'Roman name for the goddess of love. This planet was considered to be the brightest and most beautiful planet or star in the\n                    heavens. Other civilizations have named it for their god or goddess of love/war.', 'MoreInfo': 'https://en.wikipedia.org/wiki/Venus'}, {'Name': 'Earth', 'Mass': '5.97', 'Radius': '12756', 'Description': "The name Earth comes from the Indo-European base 'er,'which produced the Germanic noun 'ertho,' and ultimately German 'erde,'\n                    Dutch 'aarde,' Scandinavian 'jord,' and English 'earth.' Related forms include Greek 'eraze,' meaning\n                    'on the ground,' and Welsh 'erw,' meaning 'a piece of land.'", 'MoreInfo': 'https://en.wikipedia.org/wiki/

### **How to do it**

In [10]:
import csv
# if importing from another file
# from get_planet_data import get_planet_data

planets = get_planet_data()

with open('./artefacts/planets.csv', 'w+', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(['Name', 'Mass', 'Radius', 'Description', 'MoreInfo'])
    for planet in planets:
        writer.writerow([planet['Name'], planet['Mass'],planet['Radius'], planet['Description'], planet['MoreInfo']])

CSV data can also be read from a web server using the csv library and by first retrieving the content with requests

In [11]:
import requests
import csv

planets_data = requests.get("http://localhost:8080/planets.csv").text
planets = planets_data.split('\n')
reader = csv.reader(planets, delimiter=',', quotechar='"')
lines = [line for line in reader][:-1]
for line in lines: print(line)

['Name', 'Mass', 'Radius', 'Description', 'MoreInfo']
['Mercury', '0.330', '4879', 'Named Mercurius by the Romans because it appears to move so swiftly.', 'https://en.wikipedia.org/wiki/Mercury_(planet)']
['Venus', '4.87', '12104', 'Roman name for the goddess of love. This planet was considered to be the brightest and most beautiful planet or star in the                    heavens. Other civilizations have named it for their god or goddess of love/war.', 'https://en.wikipedia.org/wiki/Venus']
['Earth', '5.97', '12756', "The name Earth comes from the Indo-European base 'er,'which produced the Germanic noun 'ertho,' and ultimately German 'erde,'                    Dutch 'aarde,' Scandinavian 'jord,' and English 'earth.' Related forms include Greek 'eraze,' meaning                    'on the ground,' and Welsh 'erw,' meaning 'a piece of land.'", 'https://en.wikipedia.org/wiki/Earth']
['Mars', '0.642', '6792', 'Named by the Romans for their god of war because of its red, bloodlike color. O

In [13]:
# One thing to point our is that the CSV writer left a trailing blank like would add an
# empty list item if not handled. This was handled by slicing the rows: This following
# statement returned all lines except the last one: 
lines = [line for line in reader][:-1]

[]

This can also be done quite easily using pandas.

In [14]:

import pandas as pd
planets_df = pd.read_csv("http://localhost:8080/planets_pandas.csv", index_col='Name')
print(planets_df)

                                               Description       Mass  \
Name                                                                    
Mercury  Named Mercurius by the Romans because it appea...     0.3300   
Venus    Roman name for the goddess of love. This plane...     4.8700   
Earth    The name Earth comes from the Indo-European ba...     5.9700   
Mars     Named by the Romans for their god of war becau...     0.6420   
Jupiter  The largest and most massive of the planets wa...  1898.0000   
Saturn   Roman name for the Greek Cronos, father of Zeu...   568.0000   
Uranus   Several astronomers, including Flamsteed and L...    86.8000   
Neptune  Neptune was "predicted" by John Couch Adams an...   102.0000   
Pluto    Pluto was discovered at Lowell Observatory in ...     0.0146   

                                               MoreInfo  Radius  
Name                                                             
Mercury  https://en.wikipedia.org/wiki/Mercury_(planet)    4879 

And the DataFrame can be saved to a CSV file with a simple call to `.to_csv()`

In [15]:
import pandas as pd

# construct a data from from the list
planets = get_planet_data()
planets_df = pd.DataFrame(planets).set_index('Name')
planets_df.to_csv("./artefacts/planets_pandas.csv")

A CSV file can be read in from a URL very easily with `pd.read_csv()` - no need for other libraries.

In [16]:

import pandas as pd
planets_df = pd.read_csv("http://localhost:8080/planets_pandas.csv", index_col='Name')
print(planets_df)

                                               Description       Mass  \
Name                                                                    
Mercury  Named Mercurius by the Romans because it appea...     0.3300   
Venus    Roman name for the goddess of love. This plane...     4.8700   
Earth    The name Earth comes from the Indo-European ba...     5.9700   
Mars     Named by the Romans for their god of war becau...     0.6420   
Jupiter  The largest and most massive of the planets wa...  1898.0000   
Saturn   Roman name for the Greek Cronos, father of Zeu...   568.0000   
Uranus   Several astronomers, including Flamsteed and L...    86.8000   
Neptune  Neptune was "predicted" by John Couch Adams an...   102.0000   
Pluto    Pluto was discovered at Lowell Observatory in ...     0.0146   

                                               MoreInfo  Radius  
Name                                                             
Mercury  https://en.wikipedia.org/wiki/Mercury_(planet)    4879 

Converting data to JSON is also quite easy. Manipulation of JSON with Python can be done with the Python json library. This library can be used to convert Python objects to and from JSON. The following converts the list of planets into JSON and prints it to the console.

In [17]:
import json

planets = get_planet_data()
print(json.dumps(planets, indent=4))

[
    {
        "Name": "Mercury",
        "Mass": "0.330",
        "Radius": "4879",
        "Description": "Named Mercurius by the Romans because it appears to move so swiftly.",
        "MoreInfo": "https://en.wikipedia.org/wiki/Mercury_(planet)"
    },
    {
        "Name": "Venus",
        "Mass": "4.87",
        "Radius": "12104",
        "Description": "Roman name for the goddess of love. This planet was considered to be the brightest and most beautiful planet or star in the\n                    heavens. Other civilizations have named it for their god or goddess of love/war.",
        "MoreInfo": "https://en.wikipedia.org/wiki/Venus"
    },
    {
        "Name": "Earth",
        "Mass": "5.97",
        "Radius": "12756",
        "Description": "The name Earth comes from the Indo-European base 'er,'which produced the Germanic noun 'ertho,' and ultimately German 'erde,'\n                    Dutch 'aarde,' Scandinavian 'jord,' and English 'earth.' Related forms include Greek 'eraze

In [21]:
import json

planets=get_planet_data()
with open('./artefacts/planets.json', 'w+') as jsonFile:
   json.dump(planets, jsonFile, indent=4)

In [23]:
!head -n 13 ./artefacts/planets.json

[
    {
        "Name": "Mercury",
        "Mass": "0.330",
        "Radius": "4879",
        "Description": "Named Mercurius by the Romans because it appears to move so swiftly.",
        "MoreInfo": "https://en.wikipedia.org/wiki/Mercury_(planet)"
    },
    {
        "Name": "Venus",
        "Mass": "4.87",
        "Radius": "12104",
        "Description": "Roman name for the goddess of love. This planet was considered to be the brightest and most beautiful planet or star in the\n                    heavens. Other civilizations have named it for their god or goddess of love/war.",


JSON can be read from a web server with requests and converted to a Python object:

In [24]:
import requests
import json

planets_request = requests.get("http://localhost:8080/planets.json")
print(json.loads(planets_request.text))

[{'Name': 'Mercury', 'Mass': '0.330', 'Radius': '4879', 'Description': 'Named Mercurius by the Romans because it appears to move so swiftly.', 'MoreInfo': 'https://en.wikipedia.org/wiki/Mercury_(planet)'}, {'Name': 'Venus', 'Mass': '4.87', 'Radius': '12104', 'Description': 'Roman name for the goddess of love. This planet was considered to be the brightest and most beautiful planet or star in the heavens. Other civilizations have named it for their god or goddess of love/war.', 'MoreInfo': 'https://en.wikipedia.org/wiki/Venus'}, {'Name': 'Earth', 'Mass': '5.97', 'Radius': '12756', 'Description': "The name Earth comes from the Indo-European base 'er,'which produced the Germanic noun 'ertho,' and ultimately German 'erde,' Dutch 'aarde,' Scandinavian 'jord,' and English 'earth.' Related forms include Greek 'eraze,' meaning 'on the ground,' and Welsh 'erw,' meaning 'a piece of land.'", 'MoreInfo': 'https://en.wikipedia.org/wiki/Earth'}, {'Name': 'Mars', 'Mass': '0.642', 'Radius': '6792', 'D

pandas also provides JSON capabilities to save to CSV:

In [25]:
import pandas as pd

planets = get_planet_data()
planets_df = pd.DataFrame(planets).set_index('Name')
planets_df.reset_index().to_json("./artefacts/planets_pandas.json", orient='records')

Unfortunately, there is not currently a way to pretty-print the JSON that is output from `.to_json()`. Also note the use of orient='records' and the use of `rest_index()`. This is necessary for reproducing an identical JSON structure to the JSON written using the JSON library example.

JSON can be read into a DataFrame using `.read_json()`, as well as from HTTP and files

In [26]:
import pandas as pd
planets_df = pd.read_json("http://localhost:8080/planets_pandas.json").set_index('Name')
print(planets_df)

                                               Description       Mass  \
Name                                                                    
Mercury  Named Mercurius by the Romans because it appea...     0.3300   
Venus    Roman name for the goddess of love. This plane...     4.8700   
Earth    The name Earth comes from the Indo-European ba...     5.9700   
Mars     Named by the Romans for their god of war becau...     0.6420   
Jupiter  The largest and most massive of the planets wa...  1898.0000   
Saturn   Roman name for the Greek Cronos, father of Zeu...   568.0000   
Uranus   Several astronomers, including Flamsteed and L...    86.8000   
Neptune  Neptune was "predicted" by John Couch Adams an...   102.0000   
Pluto    Pluto was discovered at Lowell Observatory in ...     0.0146   

                                               MoreInfo  Radius  
Name                                                             
Mercury  https://en.wikipedia.org/wiki/Mercury_(planet)    4879 